Notebook 1.2

# NLP

El Procesamiento de Lenguaje Natural (NLP, por sus siglas en inglés) es una subárea de la inteligencia artificial que se centra en la interacción entre las computadoras y el lenguaje humano. Su aplicación en el contexto médico, especialmente en el diagnóstico con errores gramaticales, puede ofrecer diversos beneficios:

1. **Corrección Automática de Errores:** Una de las aplicaciones más directas del NLP es la corrección de errores gramaticales y ortográficos. Herramientas como TextBlob, pueden ayudar a identificar y corregir errores en el texto, aunque no son infalibles y a veces pueden introducir errores.

2. **Estandarización de Terminología:** En el ámbito médico, la terminología es crucial. Un término mal empleado o escrito incorrectamente puede cambiar significativamente el significado. El NLP puede ayudar a mapear términos mal escritos o variaciones de un término a una forma estándar.

3. **Extracción de Información:** El NLP permite extraer información relevante de textos desestructurados. Por ejemplo, si un médico escribe notas sobre un paciente, el NLP puede extraer detalles como síntomas, medicamentos prescritos y otros datos relevantes.

4. **Aumento de la Eficiencia:** Automatizar el proceso de revisión y corrección de diagnósticos mediante NLP puede ahorrar tiempo y reducir la carga de trabajo de los profesionales médicos.

5. **Mejora en la Calidad de la Atención Médica:** Un diagnóstico claro y preciso es fundamental para un tratamiento eficaz. Al reducir errores y ambigüedades en el diagnóstico, el NLP puede contribuir a una mejor atención al paciente.

6. **Minimizar Riesgos:** Errores en el diagnóstico pueden llevar a tratamientos incorrectos o a malentendidos que pueden ser perjudiciales para el paciente. Al corregir estos errores, se reduce el riesgo de posibles complicaciones.

7. **Análisis de Sentimiento y Detección de Urgencia:** El NLP puede ser utilizado para analizar el sentimiento detrás de las palabras o para detectar casos que requieran atención urgente basándose en las palabras utilizadas.

8. **Integración con Sistemas de Aprendizaje Automático:** Una vez que los datos estén limpios y estructurados, pueden ser alimentados en sistemas de aprendizaje automático para hacer predicciones o identificar patrones en los diagnósticos.

Sin embargo, es importante señalar que, aunque el NLP tiene un potencial considerable, su aplicación en el contexto médico también tiene desafíos. El lenguaje médico es complejo y a menudo requiere una comprensión profunda del contexto. Además, un error en la corrección o interpretación del NLP puede tener consecuencias graves en un entorno médico. Por lo tanto, es crucial que cualquier sistema de NLP implementado en este ámbito sea exhaustivamente probado y validado.

In [1]:
# Modulos
import pandas as pd
import numpy as np

import collections
import time
import nltk
from nltk.corpus import stopwords
from gensim.models import Word2Vec

from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

from textblob import TextBlob
from spellchecker import SpellChecker
import spacy
from spacy.language import Language
!python -m spacy download es_core_news_sm


In [2]:
%load_ext kedro.ipython

[10/23/23 15:19:12] INFO     Resolved project path as: c:\Users\Pablo                               __init__.py:139
                             Villar\Desktop\CURSOS\KSchool\Máster en Data                                          
                             Science\TFM\nuevo_entorno_kedro\TFM_HADO_Cares\hado.                                  
                             To set a different path, run '%reload_kedro <project_root>'                           

[10/23/23 15:19:13] WARNING  c:\Users\Pablo Villar\Desktop\CURSOS\KSchool\Máster en Data            warnings.py:109
                             Science\TFM\nuevo_entorno_kedro\TFM_HADO_Cares\.venv\lib\site-packages                
                             \kedro\framework\session\session.py:266: FutureWarning: ConfigLoader                  
                             will be deprecated in Kedro 0.19. Please use the OmegaConfigLoader                    
                             instead. To consult the documentation for OmegaConfigLoader, see here:                
                             https://docs.kedro.org/en/stable/configuration/advanced_configuration.                
                             html#omegaconfigloader                                                                
                               warnings.warn(                                                                      
                                                                                                                   

                    WARNING  c:\Users\Pablo Villar\Desktop\CURSOS\KSchool\Máster en Data            warnings.py:109
                             Science\TFM\nuevo_entorno_kedro\TFM_HADO_Cares\.venv\lib\site-packages                
                             \kedro\io\data_catalog.py:300: FutureWarning: Defining the 'layer'                    
                             attribute at the top level is deprecated and will be removed in Kedro                 
                             0.19.0. Please move 'layer' inside the 'metadata' -> 'kedro-viz'                      
                             attributes. See                                                                       
                             https://docs.kedro.org/en/latest/visualisation/kedro-viz_visualisation                
                             .html#visualise-layers for more information.                                          
                               warnings.warn(                                                                      
                                                                                                                   

                    INFO     Kedro project hado                                                     __init__.py:108

                    INFO     Defined global variable 'context', 'session', 'catalog' and            __init__.py:109
                             'pipelines'                                                                           

[10/23/23 15:19:14] INFO     Registered line magic 'run_viz'                                        __init__.py:115

In [3]:
df = catalog.load("hado_final")

[10/23/23 15:19:16] INFO     Loading data from 'hado_final' (CSVDataSet)...                     data_catalog.py:492

Una vez realizada la limpieza y preprocesamiento
## Hacemos la vectorización de texto

La vectorización de texto es el proceso de convertir palabras en vectores numéricos que pueden ser entendidos y procesados por un algoritmo de machine learning. Existen diferentes técnicas para hacer esto, entre las que se incluyen Bag of Words, TF-IDF, y Word2Vec.

>Word2Vec es un método que utiliza redes neuronales para aprender representaciones vectoriales de palabras a partir de un corpus de texto. Este método tiene la ventaja de que puede capturar el significado semántico de las palabras.

Tenemos que convertir los diagnosticos en una lista de listas haciendo un split de la variable 
```python
diagnosticos = df['diagnosticos'].unique()`
```

In [4]:
# Cargamos las stopwords

# import nltk
# from nltk.corpus import stopwords

# Descargar las stopwords de NLTK
nltk.download('stopwords')

# Obtener las stopwords en español
stopwords_es = stopwords.words('spanish')

# Aplicar la eliminación de stopwords a tu columna de texto
df['diagnostico'] = df['diagnostico'].apply(lambda x: ' '.join([word for word in x.split()\
                                                                if word not in (stopwords_es)]))


[nltk_data] Downloading package stopwords to C:\Users\Pablo
[nltk_data]     Villar\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [5]:
diagnosticos = df['diagnostico'].unique()
diagnosticos_split = [diagnostico.split() for diagnostico in diagnosticos]

In [6]:
diagnosticos_split[:10]


[
    ['cancer', 'orl'],
    ['melanoma', 'vulvar'],
    ['cancer', 'broncogenico'],
    ['cirrosis', 'hepatica'],
    ['neoplasia', 'mama', 'estadio', 'IV'],
    ['anemia'],
    ['adenoma', 'pulmon'],
    ['leucemia', 'linfatica', 'cronica'],
    ['fx', 'meseta', 'tibial'],
    ['cancer', 'mama']
]

In [7]:
from gensim.models import Word2Vec

# Asumiendo que tus diagnósticos ya están preprocesados y almacenados en una lista de listas,
# donde cada lista interna contiene las palabras de un diagnóstico.
# Por ejemplo: [["neoplasia", "pancreas"], ["adenocarcinoma", "de", "pulmón"], ...]

# Entrenamos el modelo Word2Vec
model = Word2Vec(diagnosticos_split, min_count=1, vector_size=50)

# Ahora podemos obtener el vector de una palabra en particular
vector = model.wv['cancer']

# O incluso obtener la palabra más similar a 'cancer'
similar = model.wv.most_similar('cancer')


In [8]:
vector


array([ 0.00053881, -0.00031167,  0.01060988,  0.01891075, -0.02099425,
       -0.01649244,  0.01638055,  0.02387845, -0.01506312, -0.00725305,
        0.01432752, -0.00611988, -0.01004254,  0.01518422, -0.01241652,
        0.00026281,  0.00694646,  0.0032353 , -0.02167222, -0.02291157,
        0.01683411,  0.01296221,  0.01991697, -0.00152752,  0.01537758,
       -0.00623267, -0.00449258,  0.01096287, -0.01761723, -0.00521478,
       -0.01417956, -0.00176219,  0.02029117, -0.0151447 , -0.00802659,
       -0.00140341,  0.01823761, -0.01312482,  0.00069714, -0.01454415,
       -0.0165098 ,  0.00914041, -0.01813373, -0.00923981,  0.00649857,
        0.00011728, -0.01785925,  0.01602187,  0.0130616 ,  0.02090717],
      dtype=float32)

In [9]:
similar


[
    ('extremidades', 0.512345552444458),
    ('dilatadaicccirrosis', 0.4648784399032593),
    ('f', 0.40824922919273376),
    ('abdominaldemencia', 0.4011690616607666),
    ('hidrocefalia', 0.383207231760025),
    ('colangitiscirrosis', 0.38253164291381836),
    ('bronquialveolar', 0.36386537551879883),
    ('cv', 0.3560919761657715),
    ('secundario', 0.35554829239845276),
    ('complicacada', 0.35340601205825806)
]

### En el código anterior, min_count es el número mínimo de veces que una palabra debe aparecer en el corpus para ser incluida en el modelo, y vector_size es la dimensión de los vectores de palabra.

Word2Vec(diagnosticos_split, min_count=1, vector_size=50): Estás entrenando un modelo Word2Vec con tus diagnósticos. La opción min_count=1 significa que las palabras que aparecen al menos una vez en tus diagnósticos serán consideradas por el modelo, y vector_size=50 define la dimensionalidad de los vectores de palabras generados por el modelo.

model.wv['cancer']: Esto te devuelve el vector de la palabra 'cancer'. Este vector es un arreglo de 50 números (porque has definido vector_size=50) que representa el significado de 'cancer' según lo aprendido por el modelo de tus diagnósticos.

model.wv.most_similar('cancer'): Esto te devuelve las palabras más similares a 'cancer' según lo aprendido por el modelo de tus diagnósticos. La similitud se mide en términos de la similitud del coseno entre los vectores de palabras.

Por favor, ten en cuenta que el modelo Word2Vec necesita una gran cantidad de datos para aprender representaciones de palabras significativas. Si solo tienes un pequeño número de diagnósticos, los vectores de palabras generados por el modelo podrían no ser muy significativos. En este caso, puedes considerar el uso de vectores de palabras pre-entrenados, como los disponibles en bibliotecas como spaCy o gensim.

In [10]:
# Filtramos los diagnósticos vacíos
diagnosticos_split = [diagnostico for diagnostico in diagnosticos_split if diagnostico]

# Luego, volvemos a calcular los vectores de diagnósticos
diagnosticos_vec = [np.mean([model.wv[word] for word in diagnostico if word in model.wv.key_to_index], axis=0) for diagnostico in diagnosticos_split]

In [11]:
from sklearn.cluster import KMeans

# Primero, necesitamos convertir nuestros diagnósticos a vectores.
# Podemos hacer esto promediando los vectores de todas las palabras en cada diagnóstico.
diagnosticos_vec = [np.mean([model.wv[word] for word in diagnostico], axis=0) for diagnostico in diagnosticos_split]

# Entonces podemos entrenar un modelo KMeans en estos vectores.
kmeans = KMeans(n_clusters=5)  # Ajusta este número a la cantidad de grupos que creas adecuada
kmeans.fit(diagnosticos_vec)

# Ahora podemos obtener las etiquetas de los grupos para cada diagnóstico
labels = kmeans.labels_

[10/23/23 15:19:31] WARNING  c:\Users\Pablo Villar\Desktop\CURSOS\KSchool\Máster en Data            warnings.py:109
                             Science\TFM\nuevo_entorno_kedro\TFM_HADO_Cares\.venv\lib\site-packages                
                             \sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of                  
                             `n_init` will change from 10 to 'auto' in 1.4. Set the value of                       
                             `n_init` explicitly to suppress the warning                                           
                               warnings.warn(                                                                      
                                                                                                                   

In [12]:
labels

array([4, 3, 4, ..., 3, 0, 3])

In [13]:
# Crea un diccionario donde las llaves son las etiquetas del cluster y los valores son listas de diagnósticos pertenecientes a ese cluster
clusters = {i: [] for i in range(5)}  # Asegúrate de cambiar el rango para que coincida con el número de clusters que estás usando

for diagnostico, label in zip(diagnosticos_split, labels):
    clusters[label].append(' '.join(diagnostico))

# Ahora puedes imprimir los diagnósticos en cada cluster
for label, diagnosticos in clusters.items():
    print(f'Cluster {label}:')
    for diagnostico in diagnosticos[:10]:
        print(f'  {diagnostico}')


Cluster 0:
  anemia
  neumonia
  infeccion urinaria fallo cardiaco
  infeccion respiratoria
  sepsis respiratoria
  infeccion urinaria
  ITU
  fractura vertebral
  epoc estadio infeccion respiratoria
  infeccion herida quirurgica
Cluster 1:
  cirrosis hepatica
  neoplasia mama estadio IV
  adenoma colon IV
  cancer colon estadio IV
  cancer lengua estadio IV
  carcinoma sigma
  cancer pulmon estadio IV
  adenocarcino colon estadio IV
  neo colon
  adenocarcinoma ovario estadio IV
Cluster 2:
  leucemia linfatica cronica
  deterioro general
  intolerancia oral
  esclerosis multiple
  smd
  deterioro cognitivo
  miocardiopatia hipertrofica
  hepatopatia cronica
  ictus
  fa valvulopatia aortica insuficiencia cardiaca
Cluster 3:
  melanoma vulvar
  adenoma pulmon
  fx meseta tibial
  tumor cerebral
  sarcoidosis
  melanoma fosa nasal
  sdown
  bocio intratoracico
  demencia fallo multiorganico
  ulceras mmii
Cluster 4:
  cancer orl
  cancer broncogenico
  cancer mama
  cancer prostata
  ca

In [14]:
for diagnostico, label in zip(diagnosticos_split, labels):
    clusters[label].append(' '.join(diagnostico))
    
    '''Está iterando sobre los diagnósticos y sus etiquetas de cluster correspondientes. 
    Para cada diagnóstico, transforma la lista de palabras en una cadena de texto (usando ' '.join(diagnostico)) 
    y la añade a la lista correspondiente en el diccionario clusters.
    
    Por ejemplo, si hay un diagnóstico que es ["neoplasia", "pancreas"] y su etiqueta de cluster es 2,
    este diagnóstico se transformaría en la cadena de texto "neoplasia pancreas" y se añadiría a la lista de diagnósticos
    del cluster 2 en el diccionario clusters. Así, al final, se crea un diccionario donde para cada cluster, 
    existe una lista de todos los diagnósticos (como cadenas de texto) que pertenecen a ese cluster'''

También puedes querer investigar las palabras más representativas de cada cluster. Esto puede darte una idea de las características que el modelo está utilizando para distinguir entre los diferentes clusters.
* Código de ejemplo para hacer eso:

In [15]:
import collections
# Para cada cluster, encuentra las 10 palabras más comunes
for label, diagnosticos in clusters.items():
    # Juntamos todos los diagnósticos en una sola lista de palabras
    palabras = [palabra for diagnostico in diagnosticos for palabra in diagnostico.split()]
    
    # Contamos las ocurrencias de cada palabra
    conteo_palabras = collections.Counter(palabras)
    
    print(f'Cluster {label}:')
    for palabra, conteo in conteo_palabras.most_common(10):
        print(f'  {palabra}: {conteo}')


Cluster 0:
  infeccion: 192
  respiratoria: 62
  urinaria: 36
  epoc: 28
  ITU: 24
  linfoma: 22
  insuficiencia: 16
  cardiaca: 14
  cognitivo: 14
  severo: 14
Cluster 1:
  IV: 262
  estadio: 258
  cancer: 100
  pulmon: 38
  neoplasia: 36
  colon: 36
  neo: 30
  carcinoma: 28
  adenocarcinoma: 28
  pancreas: 24
Cluster 2:
  insuficiencia: 126
  cronica: 92
  cardiaca: 90
  neoplasia: 84
  ulcera: 68
  cognitivo: 64
  deterioro: 60
  ITU: 48
  fragil: 32
  aguda: 30
Cluster 3:
  ulceras: 58
  demencia: 42
  sindrome: 36
  adenocarcinoma: 26
  adenoma: 24
  abdominal: 22
  intestinal: 22
  tumor: 20
  amputacion: 20
  endocarditis: 20
Cluster 4:
  cancer: 266
  epidermoide: 30
  colon: 24
  pulmon: 22
  prostata: 14
  gastrico: 14
  IV: 10
  vejiga: 8
  esofago: 8
  recto: 8


>Estos clusters pueden ser útiles para entender qué tipos de diagnósticos se agrupan juntos y podrían ayudar en tareas como el análisis de datos, la recomendación de tratamientos, o la identificación de patrones en los diagnósticos.

- Es importante recordar que estos clusters se basan en la similitud de las palabras en los diagnósticos, que no necesariamente reflejan la similitud en las condiciones médicas que representan. Por lo tanto, cualquier interpretación o aplicación de estos clusters debe hacerse con cuidado y, preferiblemente, con la validación de un experto médico.

<hr>

# Otros

## Transformación diagnosticos

### 1.Corrección ortográfica: 
Usar bibliotecas como pyspellchecker para la corrección ortográfica. Sin embargo, este proceso puede ser computacionalmente costoso y no siempre proporciona resultados perfectos, ya que la corrección ortográfica puede ser un problema muy complejo. 

Otro enfoque podría ser construir un diccionario de correcciones comunes específicas para el conjunto de datos y aplicar estas correcciones de forma manual.

### 2.Procesamiento en lotes: 
En lugar de procesar el texto de cada fila de forma individual, puedes utilizar nlp.pipe(), que es una función de SpaCy que permite procesar textos en lotes. Esto puede ser mucho más eficiente que procesar cada texto de forma individual. 
> Pero queda descartado porque haré las correcciones de forma manual

### 3.Paralelización: 
Si estás utilizando una máquina con varios núcleos de CPU, puedes paralelizar el procesamiento para hacer un uso más eficiente de tus recursos computacionales. SpaCy tiene soporte para procesamiento paralelo a través de la función nlp.pipe(), simplemente estableciendo el argumento n_process a un número mayor que 1.

### 4.Clasificación de texto: 
Una vez corregidos los errores ortográficos. Se pasa a usar técnicas de aprendizaje automático para clasificar los diagnósticos en categorías. SpaCy tiene un componente de clasificación de texto con el que puedes entrenar los datos. Alternativamente, podrían usarse otras bibliotecas de aprendizaje automático, como scikit-learn, para construir un modelo de clasificación de texto.
> Utilizaré scikit-learn

### 5.Reducción de dimensionalidad: 
Algunas técnicas, como TF-IDF o word embeddings (por ejemplo, Word2Vec, FastText o las incrustaciones de palabras de SpaCy), pueden ayudarte a reducir la dimensionalidad de los datos de texto y a extraer características útiles para la clasificación.

### 6.Entrenamiento incremental: 
Si los datos están creciendo con el tiempo, puede considerarse el uso de un modelo que permita el entrenamiento incremental, es decir, un modelo que pueda ser actualizado con nuevos datos sin tener que ser reentrenado desde cero. Algunos modelos de SpaCy admiten esto, al igual que algunos modelos en bibliotecas como scikit-learn.

In [16]:
# Hacemos un sample
df_sample = df.sample(300)

In [17]:
df_sample.shape

(300, 55)

In [18]:
df_sample.head()

,h_procedencia,hospital_category,s_procedencia,procedencia_category,diagnostico,diagnosis_category,motivo_ing,ingreso_category,motivo_alta,alta_category,...,morfina,midazolam,buscapina,haloperidol,levomepromazina,medico,ayuntamiento,year,latitude,longitude
780,clinico,Santiago,orl,Otros,cancer orl pulmon,Pulmonares y respiratorias,control sintomas,Sintomas,sin condiciones para seguimiento hado,Otros,...,0,0,0,0,0,valdes,desconocido,2018,desconocido,desconocido
3024,clinico,Santiago,mir,MIR,ITU,Otros,tratamiento antibiotico IV,Tratamientos,fin cuidados,Otros,...,0,0,0,0,0,fernandez benito,Noia,2022,42.7841408,-8.8991033
3886,no,no,no,Otros,ITU,Otros,antibioterapia IV,Tratamientos,fin cuidados,Otros,...,0,0,0,0,0,gomez buela,Santiago de Compostela,2022,42.8802346,-8.5653691
1207,clinico,Santiago,mir,MIR,infeccion herida quirurgica,Infecciones,tratamiento antibiotico IV,Tratamientos,fin cuidados,Otros,...,0,0,0,0,0,valdes,desconocido,2019,desconocido,desconocido
3312,clinico,Santiago,mir,MIR,ITU,Otros,tratamiento antibiotico IV,Tratamientos,fin cuidados,Otros,...,0,0,0,0,0,gomez buela,Santiago de Compostela,2022,42.8802346,-8.5653691


# TextBlob

* Corregir errores ortográficos (usando la biblioteca TextBlob para corrección automática):

>OJO no puede tener valores NA o de otro tipo que no sea object

In [19]:
from textblob import TextBlob
import time

l = df_sample['diagnostico']

start_time = time.time()

for x in l[:10]:
    print(f"Diagnóstico original: {x}")
    print(f"Diagnóstico corregido: {TextBlob(x).correct()}")
    print("="*25)

end_time = time.time()
execution_time = end_time - start_time
print(f"Tiempo de ejecución: {execution_time} segundos")
    

Diagnóstico original: cancer orl pulmon
Diagnóstico corregido: cancer or summon
Diagnóstico original: ITU
Diagnóstico corregido: ITU
Diagnóstico original: ITU
Diagnóstico corregido: ITU
Diagnóstico original: infeccion herida quirurgica
Diagnóstico corregido: infection herd quirurgica
Diagnóstico original: ITU
Diagnóstico corregido: ITU
Diagnóstico original: infeccion urinaria
Diagnóstico corregido: infection urinary
Diagnóstico original: cancer colon
Diagnóstico corregido: cancer colon
Diagnóstico original: adenoma pulmon estadio IV
Diagnóstico corregido: adenoma summon studio of
Diagnóstico original: neoplasia cervix
Diagnóstico corregido: neoplasm croix
Diagnóstico original: sindrome mielodisplasico
Diagnóstico corregido: syndrome mielodisplasico
Tiempo de ejecución: 1.19004225730896 segundos


In [20]:
def correct_spelling_col(df, column):
    if df[column].dtype == 'object':  # comprueba si la columna es de tipo 'object'
        df[column] = df[column].apply(lambda x: str(TextBlob(x).correct()))

In [21]:
%%time 

correct_spelling_col(df_sample, 'diagnostico')

CPU times: total: 15.4 s
Wall time: 48.1 s


In [22]:
df_sample['diagnostico']


780                                  cancer or summon
3024                                              ITU
3886                                              ITU
1207                        infection herd quirurgica
3312                                              ITU
                            ...                      
3367                                infection urinary
844     it incontinencia urinary cancer cells uterine
3780                                              ITU
744                                      cancer varix
2206                                        cirrhosis
Name: diagnostico, Length: 300, dtype: object

In [23]:
# Anteriormente teniamos el valor de ITU seguido de (infeccion del tracto urinario), para evitar paréntesis y más información simplificamos a ITU en los parámetros de "data_processing" para la columna diagnóstico
mask = df_sample['diagnostico'].str.contains(r'\(.*\)',regex=True)
filtered_values = df_sample[mask]

counts = filtered_values['diagnostico'].value_counts()
counts

Series([], Name: diagnostico, dtype: int64)

In [24]:
# TextBlob no es la panacea, devuelve muchos errores y traduce al inglés además de comerse palabras
from textblob import TextBlob

def correct_spelling(df):
    for column in df:
        if df[column].dtype == 'object':  # comprueba si la columna es de tipo 'object'
            df[column] = df[column].apply(lambda x: str(TextBlob(x).correct()))

In [25]:
%%time

correct_spelling(df_sample)

CPU times: total: 5min 38s
Wall time: 18min 20s


In [26]:
df_sample.head()

,h_procedencia,hospital_category,s_procedencia,procedencia_category,diagnostico,diagnosis_category,motivo_ing,ingreso_category,motivo_alta,alta_category,...,morfina,midazolam,buscapina,haloperidol,levomepromazina,medico,ayuntamiento,year,latitude,longitude
780,clinics,Santiago,or,Trot,cancer or summon,Pulmonares y respiratorias,control sintomas,Sintomas,sin conditions para seguimiento had,Trot,...,0,0,0,0,0,valves,desconocido,2018,desconocido,desconocido
3024,clinics,Santiago,mr,MIR,ITU,Trot,tratamiento antibiotico of,Tratamientos,in cuidados,Trot,...,0,0,0,0,0,fernandez genito,Soil,2022,42.7841408,-8.8991033
3886,no,no,no,Trot,ITU,Trot,antibioterapia of,Tratamientos,in cuidados,Trot,...,0,0,0,0,0,gomez buena,Santiago de Compostela,2022,42.8802346,-8.5653691
1207,clinics,Santiago,mr,MIR,infection herd quirurgica,Infecciones,tratamiento antibiotico of,Tratamientos,in cuidados,Trot,...,0,0,0,0,0,valves,desconocido,2019,desconocido,desconocido
3312,clinics,Santiago,mr,MIR,ITU,Trot,tratamiento antibiotico of,Tratamientos,in cuidados,Trot,...,0,0,0,0,0,gomez buena,Santiago de Compostela,2022,42.8802346,-8.5653691


## Conclusiones TextBlob

Text Blob solo para palabras en inglés.
No nos valdría para corregir ortografía en español...

# Spacy
* Textos con spacy

In [27]:
df_sample_2 = df.sample(300)

In [30]:
!python -m spacy download es_core_news_sm

  Using cached https://github.com/explosion/spacy-models/releases/download/es_core_news_sm-3.7.0/es_core_news_sm-3.7.0-py3-none-any.whl (12.9 MB)
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_sm')


In [31]:
spell = SpellChecker(language='es')  # especifica el idioma

@Language.component("spell_check")
def correct_spellings(doc):
    corrected_text = [spell.correction(word.text) if spell.correction(word.text) is not None else word.text for word in doc]
    return spacy.tokens.Doc(doc.vocab, words=corrected_text)

nlp = spacy.load('es_core_news_sm')  # carga el modelo de Spacy en español

nlp.add_pipe('spell_check', after='ner')

text = "Tus texto con errres"
doc = nlp(text)

print(doc.text)

def correct_text(text):
    doc = nlp(text)
    corrected_doc = correct_spellings(doc)
    return corrected_doc.text


Tus texto con errores 


In [32]:
%%time
df_sample_2['diagnostico_corregido'] = df_sample_2['diagnostico'].apply(correct_text)

CPU times: total: 24 s
Wall time: 1min 29s


In [33]:
df_sample_2[['diagnostico','diagnostico_corregido']][:10]

,diagnostico,diagnostico_corregido
3896,meningioma ulcera necrotica,meningioma ulcera necrótica
372,leucemialinfatica aguda,leucemialinfatica aguda
1554,insuficiencia cardiaca,insuficiencia cardiaca
3883,insuficiencia cardiaca cronica,insuficiencia cardiaca cronica
519,deterioro cognitivo,deterioro cognitivo
4000,bacteriemia,bacteriemia
141,cirrosis hepatica,cirrosis hepática
1005,demencia alt comportamiento perdida via oral,demencia alt comportamiento perdida via oral
2999,infeccion respiratoria,infección respiratoria
2619,cancer mama estadio IV,cancer mama estadio IV


In [34]:
catalog.list("hado")


[
    'hado_22',
    'hado_21',
    'hado_20',
    'hado_19',
    'hado_18',
    'hado_17',
    'strip_lower_hado_17',
    'strip_lower_hado_18',
    'strip_lower_hado_19',
    'strip_lower_hado_20',
    'strip_lower_hado_21',
    'strip_lower_hado_22',
    'hado_concat',
    'hado_clean',
    'hado_clean_na',
    'hado_barthel_cleaned',
    'hado_ps_ecog_cleaned',
    'hado_gds_fast_cleaned',
    'hado_replaced_words',
    'hado_lat_lon',
    'hado_cleaned',
    'hado_cleaned_sedation',
    'hado_cleaned_medication',
    'hado_categorized',
    'hado_final',
    'hado_encoded'
]

In [35]:
# Hemos probado con los datos en los que se ha realizado toda la limpieza
# Probaré con los datos sin los replaces pero sin NAs 'hado_clean'

df_clean = catalog.load('hado_clean_na')

[10/23/23 15:47:15] INFO     Loading data from 'hado_clean_na' (CSVDataSet)...                  data_catalog.py:492

In [36]:
df_clean['diagnostico_corregido'] = df_clean['diagnostico'].apply(correct_text)

In [37]:
df_clean[['diagnostico','diagnostico_corregido']][:10]

,diagnostico,diagnostico_corregido
0,ca orl,ca oro
1,melanoma vulvar,melanoma vulgar
2,ca broncogenico,ca broncogenico
3,cirrosis hepatica,cirrosis hepática
4,neplasia de mama estadio iv,neplasia de mama estadio iv
5,anemia,anemia
6,adenoca de pulmon,adenoma de pulmon
7,leucemia linfatica cronica,leucemia linfática cronica
8,fx meseta tibial,fe meseta tibial
9,cancer de mama,cancer de mama


## Conclusiones Spacy

No nos vale para corregir ortografía en español tampoco pero es mejor que TextBlob en resultados.



# Diccionarios

Para corregir los diagnosticos asi como otras columnas, optaremos por crear listas y diccionarios con las correcciones

In [38]:
corrections = {
    'iccbacteriemia': 'insuficiencia cardiaca cronica bacteriemia',
    'icc': 'insuficiencia cardiaca cronica',
    'ic': 'insuficiencia cardiaca',
    'insf ': 'insuficiencia',
    'i cardiaca': 'insuficiencia cardiaca',
    'icardiaca': 'insuficiencia cardiaca',
    'insufcardiaca': 'insuficiencia cardiaca',
    'insuficienciacardiaca': 'insuficiencia cardiaca',
    '1tu': 'ITU',
    'itu': 'ITU',
    'ituinfeccion': 'infeccion',
    'inf': 'infeccion',
    'infecc': 'infeccion',
    'infecion': 'infeccion',
    'infec': 'infeccion',
    'capulmon': 'cancer pulmon',
    'canc': 'cancer',
    'ca': 'cancer',
    'insuf': 'insuficiencia',
    'est': 'estadio',
    'e': 'estadio',
    'fr': 'fractura',
    'deteriorocognitivo': 'deterioro cognitivo',
    'iv': 'IV',
    'resp': 'respiratoria',
    '0steomielitis': 'osteomielitis',
    'neplasia': 'neoplasia',
    'adenoca': 'adenoma',
    'sepsisi': 'sepsis',
    'caepidermoide': 'carcinoma epidermoide',
    'ucera': 'ulcera',
    'alzheimertromboembolismo': 'alzheimer trombo embolismo',
    'deteriioro': 'deterioro',
    'miasteniabronquiectasias': 'miastenia bronquiectasia',
    'brnquiectasias': 'bronquiectasia',
    'quirurgicaarteriopatia': 'quirurgica arteriopatia',
    'respiratoriasuboclusion': 'respiratoria suboclusion',
    'respiratoriainsuf': 'respiratoria insuficiencia',
    'sistemicodeterioro': 'sistemico deterioro',
    'ituca': 'itu cancer',
    'ituincont': 'itu incontinencia',
    'ituinfec': 'itu infeccion',
    'piwe': 'pie',
    'pulm': 'pulmon',
    'urinarui': 'urinaria',
    'trtraparesia': 'tetraparesia',
    'colangiocarcinima': 'colangiocarcinoma',
    'bacteriemiadeterioro': 'bacteriemia deterioro',
    'wpidermoide': 'epidermoide',
    '(infeccion del tracto urinario)': 'ITU'
}

    # Añade tantas correcciones como necesites


In [39]:
# Busco todas las filas que contienen 'cancer' y las cuento
df_clean['diagnostico'][df_clean['diagnostico'].str.contains('cancer',case=False)].value_counts()


cancer de colon                  68
cancer de pulmon                 65
cancer de pancreas               26
cancer de prostata               18
cancer de mama                   17
                                 ..
cancer intestinal                 1
cancer de gastrico estadio iv     1
cancer de cardias                 1
cancer cavidad oral               1
cancer de recto fournier          1
Name: diagnostico, Length: 69, dtype: int64

In [40]:
# Solo palabras completas 'ca'
#\b es una aserción de límite de palabra en una expresión regular,
# lo que significa que 'ca' solo se encontrará si está rodeada de límites de palabra 
# (como espacios, signos de puntuación, el inicio del texto o el final del texto).


# Reemplaza los valores NaN con una cadena vacía
df_clean['diagnostico'] = df_clean['diagnostico'].fillna('')

df_clean['diagnostico'][df_clean['diagnostico'].str.contains(r'\bca\b', case=False, regex=True)].reset_index(drop=True).value_counts()


ca pancreas                     24
ca pulmon est iv                21
ca colon estadio iv             18
ca colon                        18
ca mama est iv                  16
                                ..
ca de pancreas                   1
ca de recto estiv                1
ca prostata ee iv                1
ca epidermoide suelo de boca     1
ca celulas de merkel             1
Name: diagnostico, Length: 198, dtype: int64

In [41]:
def correct_text(text):
    words = text.split()
    corrected_words = [corrections.get(word, word) for word in words]
    corrected_text = ' '.join(corrected_words)
    return corrected_text

In [42]:
correct_text('ca pulmon est iv')

'cancer pulmon estadio IV'

In [43]:
df_clean['diagnostico_corregido'][df_clean['diagnostico_corregido'].str.contains(r'\bca\b', case=False, regex=True)].reset_index(drop=True).value_counts()


ca pancreas                      24
ca pulmon est iv                 21
ca colon estadio iv              18
ca colon                         18
ca mama estadio iv               17
                                 ..
ca de recto esti                  1
ca prostata ee iv                 1
ca epidermoide suelo de boca      1
ca colon e iv                     1
ca celulas de merced              1
Name: diagnostico_corregido, Length: 195, dtype: int64

In [44]:

df['diagnostico'][df['diagnostico'].str.contains(r'\bca\b', case=False, regex=True)].reset_index(drop=True).value_counts()

Series([], Name: diagnostico, dtype: int64)

In [45]:
# Solo palabras completas 'cancer'
df_cancer = df[df['diagnostico'].str.contains(r'\bcancer\b', case=False, regex=True)]

In [46]:
df_cancer.shape

(806, 55)

In [47]:
df_cancer['diagnostico'].value_counts()


cancer pulmon                     94
cancer colon                      93
cancer pancreas                   57
cancer pulmon estadio IV          39
cancer mama estadio IV            33
                                  ..
cancer pulmonmieloma multiple      1
cancer prostata broncopatia cr     1
cancer cardias                     1
cancer vias biliar                 1
cancer celulas merkel              1
Name: diagnostico, Length: 195, dtype: int64

<hr>

# Convertir los datos categóricos a un formato numérico:
Para convertir las variables categóricas en numéricas, puedes utilizar la función get_dummies() de pandas para realizar la codificación one-hot o la función LabelEncoder de la biblioteca scikit-learn para la codificación ordinal. Aquí tienes un ejemplo de cómo hacer la codificación one-hot:

In [48]:
df_cancer.columns


Index(['h_procedencia', 'hospital_category', 's_procedencia',
       'procedencia_category', 'diagnostico', 'diagnosis_category',
       'motivo_ing', 'ingreso_category', 'motivo_alta', 'alta_category',
       'fecha_alta', 'ap', 'n_estancias', 'n_visitas', 'paliativo_onc_noc',
       'paliativo_no_onc_noc', 'fiebre', 'disnea', 'dolor', 'delirium',
       'sedacion', 'p_terminal', 'agonia', 'ast_anorx', 'cronico_reag',
       'trato_antibiotico', 'transfusion', 'paracentesis', 'agudo_estable',
       'toracocentesis', 'fe_iv', 'ps_ecog', 'ps_ecog_classification',
       'barthel', 'barthel_classification', 'gds_fast',
       'gds_fast_classification', 'eva_ing', 'otros_complicaciones', 'otros',
       'otros_1', 'otros_2', 'otros_category', 'categorized_combined_otros',
       'tiene_sedacion', 'morfina', 'midazolam', 'buscapina', 'haloperidol',
       'levomepromazina', 'medico', 'ayuntamiento', 'year', 'latitude',
       'longitude'],
      dtype='object')

In [49]:
df_encoded = pd.get_dummies(df_cancer, columns=['diagnostico'])

In [52]:
df_encoded.head()

,h_procedencia,hospital_category,s_procedencia,procedencia_category,diagnosis_category,motivo_ing,ingreso_category,motivo_alta,alta_category,fecha_alta,...,diagnostico_infeccion urinaria cancer rinon,diagnostico_irc cancer colon estadio IV,diagnostico_itu cancer prostata hepatopatia cronica,diagnostico_itu incontinencia urinaria cancer cuello uterino,diagnostico_mediastinitis cancer pulmon,diagnostico_neumonia cancer colon,diagnostico_recidiva cancer esofago,diagnostico_sospecha cancer rectosigmoideo,diagnostico_sospecha cancer tiroides,diagnostico_vomitos diarrea cancer colon estadio IV
0,gil casares,Santiago,upal,Unidad Paliativos,Canceres y neoplasias,control sintomas,Sintomas,reingreso,Complicaciones,unknown,...,0,0,0,0,0,0,0,0,0,0
2,clinico,Santiago,mir,MIR,Canceres y neoplasias,control evolutivo,Evaluaciones,fin cuidados,Otros,unknown,...,0,0,0,0,0,0,0,0,0,0
9,clinico,Santiago,urgencias,Urgencias,Canceres y neoplasias,control sintomas,Sintomas,exitus,Exitus,unknown,...,0,0,0,0,0,0,0,0,0,0
10,clinico,Santiago,oncologia,Oncologia,Canceres y neoplasias,dolor,Sintomas,fin cuidados,Otros,unknown,...,0,0,0,0,0,0,0,0,0,0
11,clinico,Santiago,oncologia,Oncologia,Canceres y neoplasias,dolor,Sintomas,fin cuidados,Otros,unknown,...,0,0,0,0,0,0,0,0,0,0


<hr>

#  Normalizar o estandarizar las variables numéricas:
Para normalizar o estandarizar las variables numéricas, puedes utilizar las funciones MinMaxScaler o StandardScaler de scikit-learn. Aquí tienes un ejemplo de cómo estandarizar una columna numérica:

In [53]:
df_cancer_num = df_cancer.select_dtypes(exclude='object')

scaler = StandardScaler()
for col in df_cancer_num:
    df_cancer_num[col] = scaler.fit_transform(df_cancer_num[[col]])

In [54]:
df_cancer_num.describe().T

,count,mean,std,min,25%,50%,75%,max
n_estancias,806.0,-2.644700e-17,1.000621,-0.873505,-0.622973,-0.347388,0.228836,6.842884
n_visitas,806.0,-1.763133e-17,1.000621,-0.934868,-0.567066,-0.321865,0.168538,7.769775
ps_ecog,806.0,-3.195679e-17,1.000621,-4.740700,-0.806928,0.504330,0.504330,0.504330
barthel,806.0,-4.407833e-17,1.000621,-1.166557,-0.794625,-0.422692,0.693105,2.552767
gds_fast,806.0,-2.644700e-17,1.000621,-0.199175,-0.199175,-0.199175,-0.199175,6.570370
eva_ing,806.0,0.000000e+00,1.000621,-0.320407,-0.320407,-0.320407,-0.320407,4.598606
tiene_sedacion,806.0,-4.407833e-18,1.000621,-0.668609,-0.668609,-0.668609,1.495643,1.495643
morfina,806.0,-7.052533e-17,1.000621,-0.222439,-0.222439,-0.222439,-0.222439,4.495612
midazolam,806.0,-7.052533e-17,1.000621,-0.222439,-0.222439,-0.222439,-0.222439,4.495612
buscapina,806.0,-3.526267e-17,1.000621,-0.213062,-0.213062,-0.213062,-0.213062,4.693460


<hr>

# Verificar si hay errores en los datos y corregirlos:
Durante el análisis exploratorio de datos (EDA), presta atención a las distribuciones y valores atípicos. Si encuentras errores, corrige los datos según sea necesario.

<hr>

# Comprobar si hay duplicados y decidir cómo tratarlos:
Puedes utilizar la función duplicated() de pandas para encontrar registros duplicados y la función drop_duplicates() para eliminarlos. Aquí tienes un ejemplo de cómo eliminar duplicados:

In [55]:
df.drop_duplicates(inplace=True)